In [1]:
import torch
import torchvision
import torch.optim as optim
import torch.nn as nn

import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import torch.nn.functional as F

import numpy as np
import pandas as pd

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [4]:
input_size = 28
sequence_length = 28 #time step
num_layers = 2
hidden_size = 256
num_classes = 10
batch_size = 64
num_epochs = 2
learning_rate = 0.001

In [14]:
class RNN(nn.Module):
  def __init__(self, input_size ,hidden_size, num_layers, num_classes):
    super(RNN, self).__init__()
    self.hidden_size = hidden_size
    self.num_layers = num_layers
    self.rnn = nn.GRU(input_size, hidden_size, num_layers, batch_first=True) #N * time_seq * features
    self.fc = nn.Linear(hidden_size*sequence_length,num_classes)

  def forward(self, x):
    h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)

    #forward prop
    out, _ = self.rnn(x, h0)
    out = out.reshape(out.shape[0],-1)
    out = self.fc(out)

    return out

In [15]:
train_dataset = datasets.MNIST(root='dataset/', train=True, transform = transforms.ToTensor(), download=True)
test_dataset = datasets.MNIST(root='dataset/', train=False, transform = transforms.ToTensor(), download=True)

In [16]:
train_loader = DataLoader(dataset = train_dataset, batch_size = batch_size, shuffle=True)
test_loader = DataLoader(dataset= test_dataset, batch_size = batch_size, shuffle=True)

In [17]:
model = RNN(input_size, hidden_size, num_layers, num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [18]:
for epoch in range(num_epochs):
  for batch_idx, (data,targets) in enumerate(train_loader):
    data = data.to(device=device).squeeze(1)#64,28,28
    targets = targets.to(device=device)

    scores = model(data)
    loss = criterion(scores,targets)

    optimizer.zero_grad()
    loss.backward()

    optimizer.step()

In [19]:
def check_accuaracy(loader, model):
  if loader.dataset.train:
    print("check accuracy on training data")
  else:
    print("check accuracy on test data")

  num_correct = 0
  num_samples = 0
  model.eval()

  with torch.no_grad():
    for x,y in loader:
      x = x.to(device).squeeze(1)
      y = y.to(device)

      scores = model(x)
      _, predictions = scores.max(1)
      num_correct += (predictions == y).sum()
      num_samples += predictions.size(0)

    print(f'{num_correct} / {num_samples} with accuracy \
     {float(num_correct)/float(num_samples)*100:.2f}')
  model.train()

check_accuaracy(train_loader, model)
check_accuaracy(test_loader, model)


check accuracy on training data
59143 / 60000 with accuracy      98.57
check accuracy on test data
9840 / 10000 with accuracy      98.40
